In [84]:
import cv2
import numpy as np

video = cv2.VideoCapture("test_videos/solidWhiteRight.mp4")

while True:
    ret, orig_frame = video.read()
    if not ret:
        break
    # blur image
    frame = cv2.GaussianBlur(orig_frame, (3, 3), 0)
    
    # create black image
    orig_mask = np.zeros_like(frame)
    height, width,channel = orig_mask.shape
    polygon = np.array([[(50,height),(250,300),(600,300),(1200,height)]])
    # fill with useful area
    cv2.fillPoly(orig_mask,polygon,(255,255,255))
    # add comman part of image
    bit = cv2.bitwise_and(frame, orig_mask)
    
    # convert image to HSV
    hsv = cv2.cvtColor(bit, cv2.COLOR_BGR2HSV)
    low_yellow = np.array([18, 94, 140])
    up_yellow = np.array([48, 255, 255])
    # select yellow area
    mask_yellow = cv2.inRange(hsv, low_yellow, up_yellow)
    
    #select white area
    low_white = np.array([0, 0, 220])
    up_white = np.array([255, 255, 255])
    mask_white = cv2.inRange(hsv, low_white, up_white)
    mask = cv2.bitwise_or(mask_yellow,mask_white)
    
    # select color
    ret, thresh = cv2.threshold(mask, 130, 145, cv2.THRESH_BINARY) #  you can use edges = cv2.Canny(mask, 75, 150)

    lines = cv2.HoughLinesP(thresh, 1, np.pi/180, 50, maxLineGap=50)
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(frame, (x1, y1), (x2, y2), (0, 255, 0), 5)

    cv2.imshow("frame", frame)


    if cv2.waitKey(1) == ord("q"):
        break
video.release()
cv2.destroyAllWindows()